In [5]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import random
import sklearn


from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LogisticRegression
from deap import base, creator, tools, algorithms
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
#from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
import pickle
pickle_name="GA_input_features.pickle"
with open(pickle_name, 'rb') as handle:
    GA_input_features = pickle.load(handle)
import matplotlib.pyplot as plt


# In[10]:


from  ML_initial_test import ML
train='./small/session-1-DeviceGroup-1.csv'
#train='./Aalto_BIG_train_IoTDevID.csv'
vali='./small/session-2-DeviceGroup-2.csv'
test='./small/session-1-DeviceGroup-2.csv'
fold=1
#df=ML(GA_input_features,train,vali,test,fold)


# In[38]:


from  ML_initial_test import ML
#ml_list={"BaggingClassifier":BaggingClassifier()}
ml_list={"DecisionTreeClassifier":DecisionTreeClassifier()}
model=DecisionTreeClassifier()
fold=10



# In[39]:


def FeatureMaker(f):
    selected=[]
    for i,ii in enumerate(f):
        if ii:
            selected.append(GA_input_features[i])
    return selected



def evaluate(individual):
    sum_features = np.sum(individual)
    if sum_features == 0:
        return 0.0
    else:
        sum_features = np.sum(individual)
        collector_train = []
        collector_test = []
        for k in range(0, len(individual)):
            if individual[k] == 1:
                collector_train.append(X_train.iloc[:, k])
                collector_test.append(X_test.iloc[:, k])
        X_train1 = np.transpose(np.array(collector_train))
        X_test1 = np.transpose(np.array(collector_test))
        np.nan_to_num(X_train1, copy=False)
        np.nan_to_num(y_train, copy=False)
        np.nan_to_num(X_test1, copy=False)
        np.nan_to_num(y_test, copy=False)
        clf = model.fit(X_train1, y_train)
        y_pred = clf.predict(X_test1)
        
        #f1 = accuracy_score(y_test['target'].tolist(),y_pred)
        f1 = f1_score(y_test.tolist(),y_pred, average='macro')
        #print("F1: ", f1_score(y_test['target'].tolist(),y_pred, average='macro'))
        #print("ACC: ", accuracy_score(y_test['target'].tolist(),y_pred))
        return (f1,)



def ReadCSV(features,train,vali,test):
    df = pd.read_csv(train,usecols=features)#,header=None )
    X_train =df[df.columns[0:-1]]
    #X_train=np.array(X_train)
    df[df.columns[-1]] = df[df.columns[-1]].astype('category')
    y_train=df[df.columns[-1]].cat.codes  
    
    df = pd.read_csv(vali,usecols=features)#,header=None )
    X_vali =df[df.columns[0:-1]]
    #X_test=np.array(X_test)
    df[df.columns[-1]] = df[df.columns[-1]].astype('category')
    y_vali=df[df.columns[-1]].cat.codes  
    
    df = pd.read_csv(test,usecols=features)#,header=None )
    X_test =df[df.columns[0:-1]]
    #X_test=np.array(X_test)
    df[df.columns[-1]] = df[df.columns[-1]].astype('category')
    y_test=df[df.columns[-1]].cat.codes  
    return X_train, y_train,  X_vali, y_vali, X_test, y_test


# In[41]:


file_list={"DD1":['./small/session-1-DeviceGroup-1.csv','./small/session-2-DeviceGroup-2.csv','./small/session-1-DeviceGroup-2.csv'],
"DD1T":['./small/session-2-DeviceGroup-2.csv','./small/session-1-DeviceGroup-1.csv','./small/session-2-DeviceGroup-1.csv'],
"DD2":['./small/session-2-DeviceGroup-1.csv','./small/session-1-DeviceGroup-2.csv','./small/session-2-DeviceGroup-2.csv'],
"DD2T":['./small/session-1-DeviceGroup-2.csv','./small/session-2-DeviceGroup-1.csv','./small/session-1-DeviceGroup-1.csv']}


# In[48]:




In [ ]:

final_features=[]
for i in file_list:
    features=GA_input_features
    train=file_list[i][0]
    vali=file_list[i][1]
    test=file_list[i][2]
    print(f"Train: {train}, Validation: {vali}, Test: {test}")
    X_train, y_train, X_test, y_test, X_final, y_final=ReadCSV(features,train,vali,test)
    print("Before GA")
    df=ML(GA_input_features,train,vali,test,fold,ml_list)

    # In[49]:
    
    
    # Ağırlığı 1 olan bir fitness fonksiyonu oluşturun. 
    # Eğer maksimize edilen bir fonksiyonsa ağırlık pozitif olmalıdır, 
    #aksi takdirde minimizasyon için negatif olmalıdır. Burada doğruluğu maksimize etmek istiyoruz.
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    # Fitness fonksiyonu ile tanımlanmış boş bir birey oluşturun.
    # Birey, kromozomla eşanlamlıdır
    creator.create("Individual", list, fitness=creator.FitnessMax)
    # Kromozomun boyutunu belirtin. Burada boyut değişken sayısı olacaktır
    ind_size = X_train.shape[1]
    # Toolbox'ı tanımlayın. Toolbox, tüm tanımlanan operatörleri içeren bir konteynerdir
    # Herhangi bir operatörü toolbox'a dahil etmek için kaydedilmelidir
    toolbox = base.Toolbox()
    # 1 veya 0'ı rastgele üretecek bir sayı üretici fonksiyonunu kaydedin.
    # Bu, bir değişkenin bir bireyde mevcut olup olmadığını belirtir. 1: Var; 0: Yok
    # Buradaki 1/0 değerleri alellerdir
    toolbox.register("attrib_bin", random.randint, 0, 1)
    # Yukarıda tanımlanan rastgele sayı üreteci ile bireylere değerler ekleyin
    # Her bireyin boyutu değişken sayısı olacaktır
    # Bireyleri toolbox'a kaydedin
    # initRepeat argümanı bireyi başlatmak için kullanılır
    toolbox.register(
        "individual", tools.initRepeat, creator.Individual, toolbox.attrib_bin, n=ind_size
    )
    # Yukarıda oluşturulan bireylerle bir populasyon tanımlayın
    # Popülasyonu toolbox'a kaydedin
    # initRepeat argümanı populasyonu başlatmak için kullanılır
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    
    # In[50]:
    
    
    #ind1 = toolbox.individual()
    #print("Toplam değişken sayısı/her bir kromozom uzunluğu = ", ind_size)
    #print(ind1)
    
    
    # In[51]:
    

    
    # In[52]:
    
    
    # İki nokta çaprazlama
    toolbox.register("mate", tools.cxTwoPoint)
    # Bit flip mutasyonu. indpb argümanı her bir özniteliğin ters çevrilme olasılığıdır
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.01)
    # Tournsize argümanıyla rastgele seçilen bireyler arasından en iyi bireyi seçin
    toolbox.register("select", tools.selNSGA2)
    #########################toolbox.register("select", tools.selTournament, tournsize=3)
    # Yukarıda tanımlanan uygunluk fonksiyonunu toolbox'a kaydedin
    toolbox.register("evaluate", evaluate)
    # Algoritmanın çalışması sırasında gösterilecek istatistikleri tanımlayın.
    # Her nesil için minimum, maksimum ve ortalama doğruluk seçtik
    # Kararı, daha önce belirtildiği gibi maksimum doğruluk üzerine yapacağız
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("Mean", np.mean)
    stats.register("Max", np.max)
    stats.register("Min", np.min)
    
    
    # In[53]:
    
    
    # İlk populasyonu, burada sadece gösterim için düşük bir populasyon boyutu (sadece 20) olarak tanımlıyoruz
    pop = toolbox.population(n=20)
    # HallOfFame argümanı, tüm nesiller boyunca en iyi bireyi kaydeder
    # Bu, son değişken kombinasyonumuzu belirlemek için kullanılacaktır. 
    # Sadece ilk en iyiyi alıyoruz. hof içinde k sayıda en iyi çözümü kaydetmek mümkündür
    hof = tools.HallOfFame(1)
    # Şimdi, algoritmayı DEAP'teki mevcut en basit yöntemle çalıştıralım (eaSimple).
    # Argümanlar şu şekildedir:
    # pop: Daha önce tanımlanan populasyon
    # toolbox: tüm tanımlanan operatörleri içeren toolbox
    # cxpb: İki bireyi çiftleştirmenin olasılığı.
    # mutpb: Bir bireyi değiştirmenin olasılığı. Etkiyi göstermek için yüksek tutuyoruz
    # ngen: Nesil sayısı.
    pop, log = algorithms.eaSimple(
        pop, toolbox, cxpb=0.5, mutpb=0.3, ngen=50, halloffame=hof, stats=stats
    )
    # En iyi bireyi alın
    best = hof.items[0]
    # En iyi bireyi ve onun doğruluğunu yazdırın
    #print("En İyi Birey = ", best)
    #print("En İyi Uygunluk = ", best.fitness.values[0])
    selected=FeatureMaker(best)
    selected.append("Label")
    print("\n\nAfter GA")
    df=ML(selected,train,vali,test,fold,ml_list)
    print("\n","*"*140,"\n\n\n\n\n")
    final_features.append(selected)

Train: ./small/session-1-DeviceGroup-1.csv, Validation: ./small/session-2-DeviceGroup-2.csv, Test: ./small/session-1-DeviceGroup-2.csv
Before GA


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.32s/it]


    ML                        Vali-ACC    Vali-F1    Vali-STD    Test-ACC    Test-F1    Test-STD    Total-TIME
--  ----------------------  ----------  ---------  ----------  ----------  ---------  ----------  ------------
 0  DecisionTreeClassifier      0.9153      0.441      0.0372      0.6083     0.3028      0.0686        6.3229
gen	nevals	Mean    	Max     	Min     
0  	20    	0.416829	0.541472	0.257808
1  	13    	0.425264	0.541472	0.257932
2  	15    	0.43097 	0.541472	0.318602
3  	12    	0.416795	0.541472	0.260454
4  	14    	0.407807	0.504426	0.259484
5  	17    	0.410562	0.541472	0.257298
6  	12    	0.417388	0.541472	0.301364
7  	12    	0.417365	0.521816	0.298183
8  	12    	0.410939	0.430361	0.300732
9  	14    	0.422674	0.541472	0.347807
10 	5     	0.431955	0.541472	0.347807
11 	11    	0.424279	0.504426	0.348661
12 	13    	0.428304	0.541472	0.364886
13 	11    	0.420434	0.541472	0.25738 
14 	12    	0.418249	0.541472	0.257737
15 	15    	0.421928	0.541472	0.34815 
16 	13    	0.419038	0

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.68s/it]


    ML                        Vali-ACC    Vali-F1    Vali-STD    Test-ACC    Test-F1    Test-STD    Total-TIME
--  ----------------------  ----------  ---------  ----------  ----------  ---------  ----------  ------------
 0  DecisionTreeClassifier      0.4883     0.4035      0.0028      0.4871     0.4071      0.0005         8.683

 ******************************************************************************************************************************************** 





Train: ./small/session-2-DeviceGroup-2.csv, Validation: ./small/session-1-DeviceGroup-1.csv, Test: ./small/session-2-DeviceGroup-1.csv
Before GA


...

    ML                        Vali-ACC    Vali-F1    Vali-STD    Test-ACC    Test-F1    Test-STD    Total-TIME
--  ----------------------  ----------  ---------  ----------  ----------  ---------  ----------  ------------
 0  DecisionTreeClassifier      0.9895     0.4723      0.0419      0.5561     0.2222           0         7.408
gen	nevals	Mean    	Max     	Min    
0  	20    	0.438949	0.464308	0.36398
1  	15    	0.436513	0.464282	0.366176
2  	12    	0.441158	0.464308	0.365873
3  	17    	0.439725	0.464308	0.366049
4  	15    	0.442643	0.464308	0.365695
5  	13    	0.445108	0.555556	0.365784
6  	15    	0.447011	0.555556	0.346014
7  	10    	0.451787	0.555556	0.346014
8  	12    	0.437315	0.464308	0.346014
9  	18    	0.44364 	0.555556	0.345128
10 	13    	0.428359	0.554292	0.309929
11 	12    	0.424457	0.464282	0.309929
12 	18    	0.428796	0.464308	0.310393
13 	13    	0.429648	0.464282	0.309393
14 	8     	0.433598	0.518393	0.309393
15 	9     	0.433274	0.518393	0.309393
16 	15    	0.44489 	0.5

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.78s/it]


    ML                        Vali-ACC    Vali-F1    Vali-STD    Test-ACC    Test-F1    Test-STD    Total-TIME
--  ----------------------  ----------  ---------  ----------  ----------  ---------  ----------  ------------
 0  DecisionTreeClassifier      0.6025     0.5343      0.0157      0.6277      0.562      0.0205        9.7664

 ******************************************************************************************************************************************** 





Train: ./small/session-2-DeviceGroup-1.csv, Validation: ./small/session-1-DeviceGroup-2.csv, Test: ./small/session-2-DeviceGroup-2.csv
Before GA


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.90s/it]


    ML                        Vali-ACC    Vali-F1    Vali-STD    Test-ACC    Test-F1    Test-STD    Total-TIME
--  ----------------------  ----------  ---------  ----------  ----------  ---------  ----------  ------------
 0  DecisionTreeClassifier      0.6076     0.0926      0.0029      0.5976     0.2206      0.0017        6.8908
gen	nevals	Mean     	Max      	Min      
0  	20    	0.0936813	0.0996335	0.0909091
1  	13    	0.0933145	0.0996093	0.0909049
2  	7     	0.0935233	0.100516 	0.0909049
3  	12    	0.093292 	0.0996093	0.0909049
4  	13    	0.0933699	0.0996093	0.0909049
5  	13    	0.0930053	0.0996093	0.0909049
6  	14    	0.0921375	0.0993183	0.0909049
7  	12    	0.0926841	0.099604 	0.090907 
8  	6     	0.0927205	0.099604 	0.0909049
9  	16    	0.0931814	0.099604 	0.0909049
10 	11    	0.0929235	0.0998515	0.0909049
11 	15    	0.093193 	0.0998515	0.0909049
12 	13    	0.0933368	0.0998515	0.0909049
13 	11    	0.0937843	0.0998515	0.0909049
14 	13    	0.0937173	0.101925 	0.0909049
15 	8     	

In [ ]:
final_features